# DSC 198 Soccer: Web Scraper

## Import Packages

In [1]:
#tabular formatting + functions and calculations
import pandas as pd
import numpy as np

#web scraping
import requests
from bs4 import BeautifulSoup

#misc
import os
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'../data')

import src.playbyplay as pbp

%load_ext autoreload
%autoreload 2

In [2]:
game_info = pbp.get_game_info()
games = ['Humboldt', 'Point Loma', 'Pomona', 'Sonoma']
game_list = '|'.join(games)
game_info = game_info.loc[game_info['Opponent'].str.contains(game_list)]

game_info

,Opponent,Timestamp,Suffix
7,Cal Poly Pomona,"October 4, 2019 at 4:30 PM",/sports/womens-soccer/stats/2019/cal-poly-pomo...
9,Humboldt State,"October 11, 2019 at 7:00 PM",/sports/womens-soccer/stats/2019/humboldt-stat...
14,Sonoma State,"October 25, 2019 at 12:30 PM",/sports/womens-soccer/stats/2019/sonoma-state/...
16,Cal Poly Pomona,"November 15, 2019 at 7:00 PM",/sports/womens-soccer/stats/2019/cal-poly-pomo...
17,Sonoma State,"November 17, 2019 at 3:00 PM",/sports/womens-soccer/stats/2019/sonoma-state/...
18,Point Loma,"November 23, 2019 at 2:00 PM",/sports/womens-soccer/stats/2019/point-loma/bo...


In [3]:
home_games = game_info.iloc[[0, 1, 2, 5]]
home_games

,Opponent,Timestamp,Suffix
7,Cal Poly Pomona,"October 4, 2019 at 4:30 PM",/sports/womens-soccer/stats/2019/cal-poly-pomo...
9,Humboldt State,"October 11, 2019 at 7:00 PM",/sports/womens-soccer/stats/2019/humboldt-stat...
14,Sonoma State,"October 25, 2019 at 12:30 PM",/sports/womens-soccer/stats/2019/sonoma-state/...
18,Point Loma,"November 23, 2019 at 2:00 PM",/sports/womens-soccer/stats/2019/point-loma/bo...


In [52]:
for row in range(len(home_games)):
    file = home_games.iloc[row]['Opponent'] + ' ' + home_games.iloc[row]['Timestamp'] + '.csv'

    curr_path = os.getcwd()
    parent_path = os.path.abspath(os.path.join(curr_path, os.pardir))
    data_folder = '/data/Play by Play/'
    fp = parent_path + data_folder + file
    
    play_by_play = pbp.get_play_by_play(home_games.iloc[row]['Suffix'])
    
    play_by_play.to_csv(fp, index = False)

In [29]:
home_URL = 'https://ucsdtritons.com/'
schedule_URL = 'https://ucsdtritons.com/sports/womens-soccer/schedule/2019'
box_score_ID = 'sidearm-schedule-game-links-boxscore'

In [30]:
page = requests.get(schedule_URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [31]:
box_score_suffixes = []
game_list = []

for game in soup.find_all("li", class_ = box_score_ID):
    
    try:
        suffix = game.find('a').get('href')
        game_opp = game.find('a').get('aria-label')
        
        if suffix not in box_score_suffixes:
            box_score_suffixes.append(suffix)
        
        if game_opp not in game_list:
            game_list.append(game_opp)
            
    except:
        continue

In [32]:
times = []
opps = []

for game in game_list:
    time = game.split(' on ')[1]
    times.append(time)
    
    try:
        opponent = game.split(' on ')[0].split(' vs ')[1]
        opps.append(opponent)
        
    except:
        opponent = game.split(' on ')[0].split(' at ')[1]
        opps.append(opponent)

In [33]:
df = pd.DataFrame()

df['Opponent'] = opps
df['DateTime'] = times
df['Link'] = box_score_suffixes

df.head()

,Opponent,DateTime,Link
0,Concordia (Irvine),"September 5, 2019 at 7:00 PM",/sports/womens-soccer/stats/2019/concordia-irv...
1,Biola,"September 7, 2019 at 4:30 PM",/sports/womens-soccer/stats/2019/biola/boxscor...
2,Azusa Pacific,"September 14, 2019 at 7:30 PM",/sports/womens-soccer/stats/2019/azusa-pacific...
3,Simon Fraser,"September 16, 2019 at 11:30 AM",/sports/womens-soccer/stats/2019/simon-fraser/...
4,Cal State San Marcos,"September 19, 2019 at 12:30 PM",/sports/womens-soccer/stats/2019/cal-state-san...


In [34]:
a1 = df.iloc[0]['Link']
a2 = home_URL + a1
a2

'https://ucsdtritons.com//sports/womens-soccer/stats/2019/concordia-irvine-/boxscore/2931'

In [35]:
page = requests.get(a2)
soup = BeautifulSoup(page.content, 'html.parser')

In [40]:
a = soup.find(id = "period-7")

if a:
    print('not empty')
    
b = 'period-1'

In [41]:
#soup.find(id = "period-1").find_all("td")

timer = True
timestamps = []
events = []

for table_entry in soup.find(id = b).find_all("td"):
    
    if table_entry.get('aria-hidden'):
        continue
        
    table_text = table_entry.text.strip()
    
    if len(table_text) > 2:
        
        if timer:
            timestamps.append(table_text)
            timer = False
            
        else:
            events.append(table_text)
            timer = True

In [42]:
def get_minutes(timestamp):
    return timestamp.split(':')[0]

def get_seconds(timestamp):
    return timestamp.split(':')[1]

In [43]:
event_df = pd.DataFrame()
event_df['Timestamps'] = timestamps
event_df['Event'] = events

event_df['Minute'] = event_df['Timestamps'].apply(get_minutes)
event_df['Second'] = event_df['Timestamps'].apply(get_seconds)

event_df.head()

,Timestamps,Event,Minute,Second
0,00:00,"Ramos, Angelica at goalie for UC San Diego",00,00
1,00:00,"Whitcomb, Kaelyn at goalie for Concordia (CA)",00,00
2,01:03,"Foul on Barton, Danni.",01,03
3,02:33,"Shot by UCSD Tang, Lucy, out left.",02,33
4,03:35,"Shot by UCSD Saddic, Natalie, out top.",03,35
